# IRIS TensorFI

## Dependencies

-  Python 2.7

In [1]:
!pip install -q scipy scikit-learn PyYAML matplotlib keras==2.3.0 pandas typing tensorflow==1.14.0

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
%load_ext tensorboard

In [3]:
import csv
import logging
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.metrics import categorical_accuracy
from keras.models import load_model
from keras.objectives import categorical_crossentropy
from keras.utils import to_categorical

import TensorFI as ti

Using TensorFlow backend.


In [4]:
pd.set_option('display.max_rows', 5)

## Experiments configuration

In [5]:
SEED = 0
np.random.seed(SEED)

CWD_PATH = os.getcwd()
DATASET_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/iris-data-set.csv")
MODEL_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/iris_neural_network.h5")
CONFIG_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/config/default.yaml")
LOGS_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/")

## Load Iris Dataset as categorical

In [6]:
# Load Iris dataset.
with open(DATASET_PATH, 'r') as csvfile:
    iris = list(csv.reader(csvfile))[1:]

num_input = 4
num_classes = 3

# The inputs are four floats: sepal length, sepal width, petal length, petal width.
inputs = np.array(iris)[:, :num_input].astype(
    np.float)  # We select the first three columns.

# Outputs are initially individual strings: setosa, versicolor or virginica.
outputs = np.array(iris)[:, 4]  # We select the 4th column.

# Convert the output strings to ints.
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

# Encode the category integers as binary categorical vairables.
outputs_cats = to_categorical(outputs_ints)

# Split the input and output data sets into training and test subsets.
inds = np.random.permutation(len(inputs))

train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test, outputs_test = inputs[test_inds], outputs_cats[test_inds]

## Configure Keras session to use Tensorflow Session

In [7]:
# TF-Keras Session
sess = tf.compat.v1.Session()
K.set_session(sess)

init = tf.global_variables_initializer()
sess.run(init)

## Load our Model

(Credits to Mathieu DUMONT)

Our model is :

- 4 features
- FC 5
- Activation Relu
- FC 3
- Activation Soft-Max
- 3 labels

In [8]:
# Model
K.set_learning_phase(0)
model = load_model(MODEL_PATH, custom_objects=None, compile=True)
model.summary()
x = model.input
y = model.output


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 25        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


## Evaluate the Model w/o FI

Evaluate:

In [9]:
# Evaluate model
with sess.as_default():
    y_preds = y.eval(feed_dict={x: inputs_test})
    accuracy = tf.reduce_mean(categorical_accuracy(outputs_test,
                                                   y_preds)).eval()
    loss = tf.reduce_mean(categorical_crossentropy(outputs_test,
                                                   y_preds)).eval()

Results:

In [10]:
"Loss/Accuracy (W/O FI) = " + str([loss, accuracy])

'Loss/Accuracy (W/O FI) = [0.18939355, 0.9866667]'

The predictions are (with the closest to 1.0 being the best choice):

In [11]:
pd.DataFrame(y_preds)  # Predictions

NameError: name 'output_value' is not defined

## Evaluate the model with FI

### Arm the FI

In [ ]:
fi = ti.TensorFI(sess,
                 configFileName=CONFIG_PATH,
                 logDir=LOGS_PATH,
                 logLevel=logging.INFO,
                 disableInjections=True,
                 name="IrisInjection",
                 fiPrefix="fi_")
fi.turnOnInjections()

We can observe some warnings such as `WARNING:root:operatorReadVariableOp is copied with [tf.float32] output types and [tf.resource] input types`.  This means that the operation is not supported by TensorFI and therefore has no fault injection capability.

We ignore these warnings when initializing or reading a variable (unless we want to inject fault at initialization/reading), but some operations may require fault injection capability to be implemented. In this model, 2 operations lacked fault injeciton capability :

-  `LogSoftmax`
-  `Sqrt`


### Evaluate

Before evaluating, we will store the Tensorflow graph (where all operators are located) for debugging purposes.

In [ ]:
# writer used to save graph. Use tensorboard to view.
writer = tf.compat.v1.summary.FileWriter(LOGS_PATH, sess.graph)

Evaluate :

In [ ]:
with sess.as_default():
    y_preds = y.eval(feed_dict={x: inputs_test})
    accuracy = tf.reduce_mean(categorical_accuracy(outputs_test,
                                                   y_preds)).eval()
    loss = tf.reduce_mean(categorical_crossentropy(outputs_test,
                                                   y_preds)).eval()

Results:

In [ ]:
"Loss/Accuracy (FI) = " + str([loss, accuracy])

The predictions are (with the closest to 1.0 being the best choice):

In [ ]:
pd.DataFrame(y_preds)  # Predictions

In [ ]:
%tensorboard --logdir DNN-model/Iris/ --port 8008

In [ ]:
writer.close()
sess.close()